In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.models import Model
from keras.layers import Conv2D, Input, BatchNormalization, LeakyReLU, ZeroPadding2D, UpSampling2D, AveragePooling2D
from keras.layers.merge import add, concatenate

### Create YoloV3 Model

In [72]:
def createYoloLyr(inputs, convLyrs, skip=True):
    '''Create a single YOLO layer'''
    x = inputs
    loopCounter = 0
    for convLyr in convLyrs:
        if loopCounter == (len(convLyrs)-2) and skip:
            skip_connection = x
        loopCounter += 1
        if convLyr['strides'] > 1: 
            x = ZeroPadding2D(((1,0),(1,0)), name='zeropad_'+str(convLyr['lyrName']))(x)
        x = Conv2D(filters=convLyr['filters'], 
                   kernel_size=convLyr['kernel_size'], 
                   strides=convLyr['strides'], 
                   padding='valid' if convLyr['strides']> 1 else 'same',  
                   use_bias=False if convLyr['bnorm'] else True, 
                   name='conc2d_'+str(convLyr['lyrName']))(x)
        if convLyr['bnorm']:
            x = BatchNormalization(epsilon=0.001, name='bnorm_idx_'+str(convLyr['lyrName']))(x)
        if convLyr['leakyRelu']:
            x = LeakyReLU(alpha=0.1, name='leakyrelu_idx_'+str(convLyr['lyrName']))(x)
    return add([skip_connection, x]) if skip else x

In [73]:
def createYoloBlk1(inputs):
    '''Create initial blocks'''
    x = createYoloLyr(inputs, [
        {'filters': 32, 'kernel_size': 3, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 0},
        {'filters': 64, 'kernel_size': 3, 'strides': 2, 'bnorm': True, 'leakyRelu': True, 'lyrName': 1},
        {'filters': 32, 'kernel_size': 1, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 2},
        {'filters': 64, 'kernel_size': 3, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 3},
    ])
    x = createYoloLyr(x, [
        {'filters': 128, 'kernel_size': 3, 'strides': 2, 'bnorm': True, 'leakyRelu': True, 'lyrName': 5}, # Index jumped to 5 from 3 because of additional add layer
        {'filters': 64, 'kernel_size': 1, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 6},
        {'filters': 128, 'kernel_size': 3, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 7},
    ])
    x = createYoloLyr(x, [
        {'filters': 64, 'kernel_size': 1, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 9},
        {'filters': 128, 'kernel_size': 3, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 10},
    ])
    return x

In [74]:
def createYoloBlk2(inputs): # Block for small objects detection
    '''Create block for small objects detection'''
    x = createYoloLyr(inputs, [
        {'filters': 256, 'kernel_size': 3, 'strides': 2, 'bnorm': True, 'leakyRelu': True, 'lyrName': 12},
        {'filters': 128, 'kernel_size': 1, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 13},
        {'filters': 256, 'kernel_size': 3, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 14},
    ])
    for idx in range(7):
        x = createYoloLyr(x, [
            {'filters': 128, 'kernel_size': 1, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 15+idx*3}, # Index * 3 is because there are 2 conv2D layer + 1 add layer
            {'filters': 256, 'kernel_size': 3, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 16+idx*3},
        ])
    return x

In [75]:
def createYoloBlk3(inputs): # Block for medium objects detection    
    '''Create block for medium objects detection'''
    x = createYoloLyr(inputs, [
        {'filters': 512, 'kernel_size': 3, 'strides': 2, 'bnorm': True, 'leakyRelu': True, 'lyrName': 37},
        {'filters': 256, 'kernel_size': 1, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 38},
        {'filters': 512, 'kernel_size': 3, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 39},
    ])
    for idx in range(7):
        x = createYoloLyr(x, [
            {'filters': 256, 'kernel_size': 1, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 41+idx*3},
            {'filters': 512, 'kernel_size': 3, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 42+idx*3},
        ])
    return x

In [76]:
def createYoloBlk4(inputs):
    '''Create block for big objects detection'''
    x = createYoloLyr(inputs, [
        {'filters': 1024, 'kernel_size': 3, 'strides': 2, 'bnorm': True, 'leakyRelu': True, 'lyrName': 62},
        {'filters': 512, 'kernel_size': 1, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 63},
        {'filters': 1024, 'kernel_size': 3, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 64},
    ])
    for idx in range(7):
        x = createYoloLyr(x, [
            {'filters': 512, 'kernel_size': 1, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 66+idx*3},
            {'filters': 1024, 'kernel_size': 3, 'strides': 1, 'bnorm': True, 'leakyRelu': True, 'lyrName': 67+idx*3},
        ])
    return x

In [77]:
def build_YoloV3_Model():
    '''Build full YoloV3 model'''
    # Input Layer
    input_img = Input(shape=(None, None, 3))
    x = createYoloBlk1(input_img)
    x = createYoloBlk2(x)
    x = createYoloBlk3(x)
    x = createYoloBlk4(x)
    x = AvgP
    return Model(input_img, x)

In [78]:
model = build_YoloV3_Model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conc2d_0 (Conv2D)               (None, None, None, 3 864         input_20[0][0]                   
__________________________________________________________________________________________________
bnorm_idx_0 (BatchNormalization (None, None, None, 3 128         conc2d_0[0][0]                   
__________________________________________________________________________________________________
leakyrelu_idx_0 (LeakyReLU)     (None, None, None, 3 0           bnorm_idx_0[0][0]                
__________________________________________________________________________________________________
zeropad_1 